In [1]:
using Pkg
Pkg.instantiate()
using DotEnv
include("src/test_name.jl")
using .Test_name

┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.to_rcm_time :: Tuple{String}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.bounding_box_to_geojson :: Tuple{}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.coordinate_to_geojson :: Union{Tuple{T}, Tuple{T, T}} where T<:Real` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.wkt_to_tuples :: Tuple{Any}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243


In [2]:
using PyCall
DotEnv.config();

In [3]:
# Define the base URL for the Sentinel-1 data collection
start_time = Test_name.Download.Metadata.to_rcm_time("2022-10-10");
end_time = Test_name.Download.Metadata.to_rcm_time("2022-10-11");
box = "70,75,5,10"
split_strings = split(box, ",")
lat_min, lat_max, lon_min, lon_max  = parse.(Int, split_strings)
#lat_min, lat_max, lon_min, lon_max = map(float, bbox.split(','))
area = Test_name.Download.Metadata.bounding_box_to_geojson(lat_min=lat_min, lat_max=lat_max, lon_min=lon_min, lon_max=lon_max);
area = Test_name.Download.Metadata.coordinate_to_geojson(55.0,10.0)

1-element Vector{Tuple{String, Vector{Tuple{Float64, Float64}}}}:
 ("contains", [(9.99, 55.01), (10.01, 55.01), (10.01, 54.99), (9.99, 54.99), (9.99, 55.01)])

In [4]:
rcm_data = Test_name.Download.Metadata.download_metadata_rcm(area = area, username = get(ENV, "RCM_USERNAME", ""), password = get(ENV, "RCM_PASSWORD", ""));

| EODMSRAPI | 2023-11-28 09:42:45 | Getting Collection information, please wait...
| EODMSRAPI | 2023-11-28 09:42:50 |  WARNING: No available field named 'Beam Mode'.

| EODMSRAPI | 2023-11-28 09:42:50 | Searching for RCMImageProducts images on RAPI
| EODMSRAPI | 2023-11-28 09:42:50 | Querying records within 1 to 1000...
| EODMSRAPI | 2023-11-28 09:42:52 | Number of RCMImageProducts images returned from RAPI: 10


| EODMSRAPI | Fetching result metadata: 100%|##########| 10/10 [00:01<00:00,  5.28item/s]


In [5]:
df = Test_name.Download.Metadata.vector_of_dicts_to_dataframe(rcm_data);

In [39]:
println(df)

10×111 DataFrame
 Row │ satellite  productDatum  sampledPixelSpacing  providerId  collectionId      datasetId  processorVersion  metadataFullUrl                    sensorId  sensorFolder  acquisitionEndDate         metadataFullName                   sipSize  geometry_crs_properties_name  creationDate               downlinkSegmentId  overviewUrl                        sampleType          inter-polarizationMatricesCorrection  lutApplied  dopplerAnomalyGrid  lookOrientation  archiveVisibilityStartDate  thisRecordUrl                      productMapZone  relativeOrbit  beamModeVersion  securityClassification        collectionTitle     isGeorectified  calibrationIssues  geometry_coordinates_7  metadata     absoluteOrbit  recordId  geometry_coordinates_1  productFormat  orderExecuteUrl                    productMapProjection  vapEnabled  processingPriority  geometry_coordinates_5  orbitDataFilename                  options  supplierOrderNumber                publicGood  numberOfAzimuthLooks  

In [7]:
path = "/Users/kaaso/Documents/coding/JuliaEO2024/test_folder_del2"

"/Users/kaaso/Documents/coding/JuliaEO2024/test_folder_del2"

UndefVarError: UndefVarError: `df` not defined

In [9]:
make_directory_recursive(path)

UndefVarError: UndefVarError: `make_directory_recursive` not defined

In [ ]:
#download_product_rcm(username="Aalling93", password = "Security1234!",product_result = res[1],destination = path)

In [12]:
function make_directory_recursive(path::String)
    # Use splitpath for platform independence
    path_parts = splitpath(path)
    
    # Handle absolute paths
    partial_path = startswith(path, "/") ? "/" : ""

    # Iterate through parts of the path
    for part in path_parts
        # Update the partial path
        partial_path = joinpath(partial_path, part)

        # Check if this part of the path exists
        if !isdir(partial_path)
            # Create the directory if it doesn't exist
            mkdir(partial_path)
        end
    end
end


make_directory_recursive (generic function with 1 method)

In [ ]:
@pyimport eodms_rapi
""" 

# priority : Medium, High, Low, Urgent

""" 
function order_product_rcm(; username::String, password::String, product_result, destination::String, priority::String="Medium") 
    # 
    #making a dir for the download   
    rapi = eodms_rapi.EODMSRAPI(username, password)
    # first, we have to order the product, and make it available for download (very similar to the studid LTR for Sentinel1...)
    order_res = rapi.order(product_result)
    rapi.clear_results()
    return order_res 
end



In [ ]:
@pyimport eodms_rapi
function get_order_status(; username::String, password::String, product_result, destination::String) 
    #making a dir for the download
    make_directory_recursive(destination)    
    rapi = eodms_rapi.EODMSRAPI(username, password)
    # first, we have to order the product, and make it available for download (very similar to the studid LTR for Sentinel1...)
    # Only when they are available for download, can we download them...
    #
    #println(param_res)
    #println("ok")
    #download = rapi.download(product_result, destination)
    statuses = []
    for product in product_result
        status = rapi.get_order_parameters("RCMImageProducts", product_result)
        push!(statuses, status)
    end

    rapi.clear_results()
    return order_res 
end


In [43]:
#order_res  = order_product_rcm(username="Aalling93", password = "Security1234!",product_result = rcm_data[7:9], destination = path);

In [63]:
order_res

Dict{Any, Any} with 1 entry:
  "items" => Dict{Any, Any}[Dict("itemId"=>"3913099", "status"=>"SUBMITTED", "d…

In [65]:
download_product_rcm(username="Aalling93", password = "Security1234!",product_result = order_res, destination = path);

"3913099"

In [45]:
Test_name.Download.Metadata.flatten_dict(order_res)

Dict{String, Any} with 18 entries:
  "items_3_itemId"          => "3913101"
  "items_1_collectionId"    => "RCMImageProducts"
  "items_1_orderId"         => "338915"
  "items_1_itemId"          => "3913099"
  "items_3_recordId"        => "14068774"
  "items_2_itemId"          => "3913100"
  "items_1_recordId"        => "14068916"
  "items_2_recordId"        => "14068858"
  "items_1_status"          => "SUBMITTED"
  "items_2_collectionId"    => "RCMImageProducts"
  "items_3_status"          => "SUBMITTED"
  "items_2_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"
  "items_2_orderId"         => "338915"
  "items_3_orderId"         => "338915"
  "items_2_status"          => "SUBMITTED"
  "items_1_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"
  "items_3_collectionId"    => "RCMImageProducts"
  "items_3_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"

In [64]:
@pyimport eodms_rapi
function download_product_rcm(; username::String, password::String, product_result, destination::String) 
    #making a dir for the download
    make_directory_recursive(destination)    
    rapi = eodms_rapi.EODMSRAPI(username, password)
    download = rapi.download(product_result, destination)
    rapi.clear_results()
    return nothing 
end



download_product_rcm (generic function with 1 method)

In [62]:
order(order_res["items"][1]["itemId"])

UndefVarError: UndefVarError: `capitalize` not defined

In [49]:
items = Test_name.Download.Metadata.vector_of_dicts_to_dataframe(order_res["items"]);

In [50]:
println(items)

3×6 DataFrame
 Row │ itemId   status     dateRapiOrdered                   collectionId      recordId  orderId 
     │ String   String     String                            String            String    String  
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │ 3913099  SUBMITTED  2023-11-28T10:09:01.454151+01:00  RCMImageProducts  14068916  338915
   2 │ 3913100  SUBMITTED  2023-11-28T10:09:01.454151+01:00  RCMImageProducts  14068858  338915
   3 │ 3913101  SUBMITTED  2023-11-28T10:09:01.454151+01:00  RCMImageProducts  14068774  338915


In [46]:
Test_name.Download.Metadata.flatten_dict(order_res)

Dict{String, Any} with 18 entries:
  "items_3_itemId"          => "3913101"
  "items_1_collectionId"    => "RCMImageProducts"
  "items_1_orderId"         => "338915"
  "items_1_itemId"          => "3913099"
  "items_3_recordId"        => "14068774"
  "items_2_itemId"          => "3913100"
  "items_1_recordId"        => "14068916"
  "items_2_recordId"        => "14068858"
  "items_1_status"          => "SUBMITTED"
  "items_2_collectionId"    => "RCMImageProducts"
  "items_3_status"          => "SUBMITTED"
  "items_2_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"
  "items_2_orderId"         => "338915"
  "items_3_orderId"         => "338915"
  "items_2_status"          => "SUBMITTED"
  "items_1_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"
  "items_3_collectionId"    => "RCMImageProducts"
  "items_3_dateRapiOrdered" => "2023-11-28T10:09:01.454151+01:00"

download_product_rcm (generic function with 1 method)

In [34]:
orders = "331361"
orders = "14068734"

"14068734"

In [35]:
r = download_product_rcm(username="Aalling93", password = "Security1234!",product_result = orders, destination = path);
Test_name.Download.Metadata.flatten_dict(r)

Dict{Any, Any}("parameters" => Dict{Any, Any}[Dict("internalName" => "packagingFormat", "allowMultiple" => false, "choices" => Dict{Any, Any}[Dict("systemDefault" => true, "title" => "Zip", "value" => "ZIP", "userDefault" => false), Dict("systemDefault" => true, "title" => "Zip", "value" => "ZIP", "userDefault" => false), Dict("systemDefault" => false, "title" => "Tar", "value" => "TAR", "userDefault" => false), Dict("systemDefault" => false, "title" => "Tar/GZ", "value" => "TARGZ", "userDefault" => false)], "title" => "Packaging Format", "description" => "Packaging Format", "type" => "choice"), Dict("internalName" => "NO_PARAMS_AVAILABLE", "allowMultiple" => false, "choices" => Dict{Any, Any}[Dict("systemDefault" => false, "title" => "No Notification", "value" => "", "userDefault" => false), Dict("systemDefault" => true, "title" => "kaaso@space.dtu.dk", "value" => "kaaso@space.dtu.dk", "userDefault" => false)], "title" => "Notification Email", "description" => "Notification email will

Dict{String, Any} with 35 entries:
  "parameters_1_choices_4_userDefault"   => false
  "parameters_2_choices_1_systemDefault" => false
  "parameters_2_choices_2_systemDefault" => true
  "parameters_1_choices_2_title"         => "Zip"
  "parameters_1_choices_4_systemDefault" => false
  "parameters_1_choices_4_value"         => "TARGZ"
  "parameters_2_allowMultiple"           => false
  "parameters_1_choices_1_title"         => "Zip"
  "parameters_1_choices_2_userDefault"   => false
  "parameters_2_choices_1_value"         => ""
  "parameters_1_allowMultiple"           => false
  "parameters_1_choices_1_value"         => "ZIP"
  "parameters_2_type"                    => "choice"
  "parameters_1_choices_2_value"         => "ZIP"
  "parameters_1_internalName"            => "packagingFormat"
  "parameters_1_choices_3_userDefault"   => false
  "parameters_1_choices_3_systemDefault" => false
  "parameters_2_title"                   => "Notification Email"
  "parameters_1_choices_3_value"     

In [37]:
println(DataFrame(Test_name.Download.Metadata.flatten_dict(r)))

1×35 DataFrame
 Row │ parameters_1_allowMultiple  parameters_1_choices_1_systemDefault  parameters_1_choices_1_title  parameters_1_choices_1_userDefault  parameters_1_choices_1_value  parameters_1_choices_2_systemDefault  parameters_1_choices_2_title  parameters_1_choices_2_userDefault  parameters_1_choices_2_value  parameters_1_choices_3_systemDefault  parameters_1_choices_3_title  parameters_1_choices_3_userDefault  parameters_1_choices_3_value  parameters_1_choices_4_systemDefault  parameters_1_choices_4_title  parameters_1_choices_4_userDefault  parameters_1_choices_4_value  parameters_1_description  parameters_1_internalName  parameters_1_title  parameters_1_type  parameters_2_allowMultiple  parameters_2_choices_1_systemDefault  parameters_2_choices_1_title  parameters_2_choices_1_userDefault  parameters_2_choices_1_value  parameters_2_choices_2_systemDefault  parameters_2_choices_2_title  parameters_2_choices_2_userDefault  parameters_2_choices_2_value  parameters_2_description  

In [ ]:
    def _extract_download_metadata(self, item):
        '''
        Because the download link in the response from EODMS is HTML-encoded, we have to parse out
        the actual download URL and the filesize

        Inputs:
          - item: JSON (dict) of item metadata from EODMS

        Outputs:
          - url: remote file URL
          - fsize: remote filesize in bytes
        '''
        # download url
        parser = EODMSHTMLFilter()
        parser.feed(item['destinations'][0]['stringValue'])
        url = parser.text
        # strip &file= from end of url
        # TODO: THIS IS A BANDAID FIX THAT WILL PROBABLY HAVE TO BE REMOVED LATER
        url = url.split('&file=')[0]
        # remote filesize
        manifest_key = list(item['manifest'].keys()).pop()
        fsize = int(item['manifest'][manifest_key])
        return url, fsize

In [ ]:
# Function to convert a vector of dictionaries to a DataFrame
function vector_of_dicts_to_dataframe(vec_of_dicts::Vector)
    rows = [flatten_dict(d) for d in vec_of_dicts]
    return DataFrame(rows)
end

In [ ]:
params = Dict("packagingFormat": "TAR")

In [17]:
download_product_rcm(username="Aalling93", password = "Security1234!",product_result = orders, destination = path)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/kaaso/.julia/packages/PyCall/KLzIO/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError('string indices must be integers')
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sar_ais_2/lib/python3.9/site-packages/eodms_rapi/eodms.py", line 1539, in download
    unique_items = self.remove_duplicate_orders(items)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sar_ais_2/lib/python3.9/site-packages/eodms_rapi/eodms.py", line 2993, in remove_duplicate_orders
    rec_ids = [o['recordId'] for o in orders]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sar_ais_2/lib/python3.9/site-packages/eodms_rapi/eodms.py", line 2993, in <listcomp>
    rec_ids = [o['recordId'] for o in orders]


In [12]:
JSON = """
{
    "destinations": [],
    "items": [
        {
            "collectionId": "RCMImageProducts",
            "recordId": "7822244",
            "parameters": [
                {
                    "packagingFormat": "TAR"
                },
                {
                    "NOTIFICATION_EMAIL_ADDRESS": "example@email.com"
                }
            ]
        }
    ]
}
"""

"{\n    \"destinations\": [],\n    \"items\": [\n        {\n            \"collectionId\": \"RCMImageProducts\",\n            \"recordId\": \"7822244\",\n            \"parameters\": [\n                {\n                    \"packagingFormat\": \"TAR\"\n                },\n                {\n                    \"NOTIFICATION_EMAIL_ADDRESS\": \"example@email.com\"\n                }\n            ]\n        }\n    ]\n}\n"

In [ ]:
#::Dict{Any, Any}